# Clash of Clans API Exploration

This notebook has some basic API exploration and the methods to fetch clan war data from the API

The API supports Bearer-style authentication which is straight forward to use with requests. A token can be generated here https://developer.clashofclans.com/#/. You need to configure all the IP adresses you want to use the API with in the CoC developer portal

In [7]:


import requests
import os

header = {"Authorization": f"Bearer {os.getenv('CLASH_OF_CLANS_KEY')}"}

mytag = '#Q8P2JR020'
myclan = '#2L8L9RLYU'

url = "https://api.clashofclans.com/v1"


The first interesting endpoint is about clanwars. We can get information about the current clan war for our own clan and all clans with public clan war log.

In [2]:
# clans = clans_response.json()['items']

# for clan in clans:
# response = requests.get(f"{url}/clans/{myclan}/currentwar", headers=header)
def get_war_log(clan_tag):
    response = requests.get(f"https://api.clashofclans.com/v1/clans/%23{clan_tag.replace('#', '')}/currentwar", headers=header)
    
    # print(response.json())
    if 'reason' in response.json().keys():
        if response.json()['reason'] == 'accessDenied':
            print("Access Denied")
            return '', []
    if 'state' in response.json().keys():
        if response.json()['state'] == 'notInWar':
            print("Not in war")
            return '', []
        if response.json()['state'] == 'preparation':
            print("In preparation")
            return '', []
    attacks = []
    # print(response.json())
    for member in response.json()['clan']['members']:
        if 'attacks' not in member.keys():
            continue
        for attack in member['attacks']:
            attacker_info = get_player_info(attack['attackerTag'])
            for key in attacker_info.keys():
                new_key = f"attacker_{key}"
                attack[new_key] = attacker_info[key]
            defender_info = get_player_info(attack['defenderTag'])
            for key in defender_info.keys():
                new_key = f"defender_{key}"
                attack[new_key] = defender_info[key]
            attacks.append(attack)
    for member in response.json()['opponent']['members']:
        if 'attacks' not in member.keys():
            continue
        for attack in member['attacks']:
            attacker_info = get_player_info(attack['attackerTag'])
            for key in attacker_info.keys():
                new_key = f"attacker_{key}"
                attack[new_key] = attacker_info[key]
            defender_info = get_player_info(attack['defenderTag'])
            for key in defender_info.keys():
                new_key = f"defender_{key}"
                attack[new_key] = defender_info[key]
            attacks.append(attack)
    attacks = sorted(attacks, key = lambda x: x.get('order'))
    # for attack in attacks:
    #    print(attack)
    clan_tags = '_'.join(sorted([response.json()['clan']['tag'], response.json()['opponent']['tag']])).replace('#', '')
    war_tag = response.json()['startTime'] + '_' + clan_tags
    return war_tag, attacks
# for line in get_war_log('#2Q9GVOYLV'):
#    print(line)

In [12]:
def get_player_info(player_tag):
    response = requests.get(f"https://api.clashofclans.com/v1/players/%23{player_tag.replace('#', '')}", headers=header)
    player_info = dict()
    for key in ["townHallLevel", "expLevel", "trophies", "bestTrophies", "warStars", "attackWins", "defenseWins", "builderHallLevel", "versusTrophies", "bestVersusTrophies", "versusBattleWins", "warPreference", "donations", "donationsReceived", "clanCapitalContributions"]: 
        player_info[key] = response.json()[key]
    if "league" in response.json().keys():
        player_info["league"] = response.json()["league"]["name"]
    else:
        player_info["league"] = "none"
    for troop in response.json()['troops']:
        key = f"troop_{troop['name'].replace(' ', '_')}_level"
        player_info[key] = troop['level']
    for hero in response.json()['heroes']:
        key = f"hero_{hero['name'].replace(' ', '_')}_level"
        player_info[key] = hero['level']
    for spell in response.json()['spells']:
        key = f"spell_{spell['name'].replace(' ', '_')}_level"
        player_info[key] = spell['level']
    return player_info

info = get_player_info(mytag)

In [6]:
response = requests.get(f"https://api.clashofclans.com/v1/players/%23{mytag.replace('#', '')}", headers=header)
response.json()

{'reason': 'accessDenied.invalidIp',
 'message': 'Invalid authorization: API key does not allow access from IP 45.14.97.10'}

In [13]:
print(f"{info['attackWins']}")

7


In [25]:
import urllib.parse
import random
import string

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.printable
    result_str = ''.join(random.choice(letters) for i in range(length))
    return urllib.parse.quote(result_str)
    
get_random_string(3)

'%24cw'

In [17]:
clans_response.json()["items"][3]

{'tag': '#2YLVL0QPQ',
 'name': 'knn',
 'type': 'open',
 'badgeUrls': {'small': 'https://api-assets.clashofclans.com/badges/70/BIZ4D9ihzimhDQ4y8GKb0VbXvJKDiLVIJ_MyIQxDVLA.png',
  'large': 'https://api-assets.clashofclans.com/badges/512/BIZ4D9ihzimhDQ4y8GKb0VbXvJKDiLVIJ_MyIQxDVLA.png',
  'medium': 'https://api-assets.clashofclans.com/badges/200/BIZ4D9ihzimhDQ4y8GKb0VbXvJKDiLVIJ_MyIQxDVLA.png'},
 'clanLevel': 1,
 'clanPoints': 1570,
 'clanVersusPoints': 1671,
 'requiredTrophies': 0,
 'warFrequency': 'unknown',
 'warWinStreak': 0,
 'warWins': 0,
 'warTies': 0,
 'warLosses': 0,
 'isWarLogPublic': True,
 'warLeague': {'id': 48000000, 'name': 'Unranked'},
 'members': 3,
 'labels': [],
 'requiredVersusTrophies': 0,
 'requiredTownhallLevel': 1}

In [26]:
import time
import datetime
import json

while True:
    random_string = get_random_string(3)
    print(f"new search string {random_string}")
    clans_response = requests.get(f"{url}/clans?name={random_string}&minMembers=20", headers=header)
    print(f"found {len(clans_response.json()['items'])} clans")
    for clan in clans_response.json()['items']:
        print(f"Parsing clan {clan['name']}")
        if not clan['isWarLogPublic']:
            print(f"Clan war log is not public")
            continue
        war_tag, attacks = get_war_log(clan['tag'])
        if len(attacks) == 0:
            continue
        print(f"{datetime.datetime.now()} - {war_tag} / {len(attacks)}")
        with open(f"clanwars/{war_tag}.json", "w") as f:
            f.write(json.dumps(attacks))
        time.sleep(5)
    

new search string ~nV
found 0 clans
new search string %28U%3C
found 0 clans
new search string o%3E0
found 0 clans
new search string 5jl
found 0 clans
new search string .uw
found 1 clans
Parsing clan .UwU.
Clan war log is not public
new search string %28%296
found 0 clans
new search string 9%3Fk
found 0 clans
new search string Xi7
found 0 clans
new search string Peu
found 57 clans
Parsing clan comme on peu
Clan war log is not public
Parsing clan curieux un peu
2022-11-26 14:00:58.407902 - 20221123T185024.000Z_2LV80QJL2_2QY9L0LVU / 17
Parsing clan Le Petit Peuple
In preparation
Parsing clan From Peuteuy
In preparation
Parsing clan le Peuple Libre
Clan war log is not public
Parsing clan peugeot 27
In preparation
Parsing clan Les ptit peudeu
In preparation
Parsing clan Meme pas peur
Clan war log is not public
Parsing clan peuyeum
In preparation
Parsing clan Même pas peur !
2022-11-26 14:03:18.694564 - 20221124T212954.000Z_G9JL29PR_GGUP2CLQ / 57


KeyboardInterrupt: 

In [55]:
clans_response.json()['paging']

{'cursors': {}}

In [13]:
clan['tag']

'#2YPRRQYR9'

In [14]:
clan

{'tag': '#2YPRRQYR9',
 'name': 'ABC',
 'type': 'inviteOnly',
 'badgeUrls': {'small': 'https://api-assets.clashofclans.com/badges/70/Mf3kGgTCa4AaGxp9ETmuOwGGTw7zpO2AlyzWKF9DpXw.png',
  'large': 'https://api-assets.clashofclans.com/badges/512/Mf3kGgTCa4AaGxp9ETmuOwGGTw7zpO2AlyzWKF9DpXw.png',
  'medium': 'https://api-assets.clashofclans.com/badges/200/Mf3kGgTCa4AaGxp9ETmuOwGGTw7zpO2AlyzWKF9DpXw.png'},
 'clanLevel': 7,
 'clanPoints': 17477,
 'clanVersusPoints': 15617,
 'requiredTrophies': 200,
 'warFrequency': 'always',
 'warWinStreak': 1,
 'warWins': 32,
 'isWarLogPublic': False,
 'warLeague': {'id': 48000008, 'name': 'Gold League II'},
 'members': 22,
 'labels': [{'id': 56000000,
   'name': 'Clan Wars',
   'iconUrls': {'small': 'https://api-assets.clashofclans.com/labels/64/lXaIuoTlfoNOY5fKcQGeT57apz1KFWkN9-raxqIlMbE.png',
    'medium': 'https://api-assets.clashofclans.com/labels/128/lXaIuoTlfoNOY5fKcQGeT57apz1KFWkN9-raxqIlMbE.png'}},
  {'id': 56000001,
   'name': 'Clan War League',
   '